In [16]:
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob
from tqdm import tqdm
import numpy as np
import math
%matplotlib inline

# 2004, 2008, 2011

import warnings
warnings.filterwarnings("ignore")

### IPEDs

In [17]:
drive_url = "/Users/fsultan_1/Library/CloudStorage/GoogleDrive-syedfahadsultan@gmail.com/My Drive/data/"

sc_url    = drive_url+"CollegeScorecard_data"
ipeds_url = drive_url+"IPEDS_data"

fnames  = glob(ipeds_url+'/completions/*_A/*_rv.csv')+glob(ipeds_url+'/completions/*_a.csv')

# unis    = pd.read_csv('../data/hd2020.csv', encoding='cp1252')
# fnames  = glob('../data/IPEDS/*_A/*_rv.csv')+glob('../data/IPEDS/*_a.csv')
all_dfs     = []

for fname in tqdm(fnames): 

    year = int(fname.split('/')[-1].split('_')[0][1:])
    if year <= 2001:
        continue 

    df          = pd.read_csv(fname, index_col=0)
    df          = df.rename(columns={col: col.upper().strip() for col in df.columns})
    df          = df[df['AWLEVEL'].apply(lambda x: x in [5, 7, 9, 17, 18, 19])]
    df['cs']    = df['CIPCODE'].apply(lambda x: (x>=11) & (x<12))
    df['year']  = year
    df['ugd']   = df['AWLEVEL'] == 5
    df['mst']   = df['AWLEVEL'] == 7
    df['phd']   = df['AWLEVEL'] >  7

    all_dfs.append(df)

all_dfs = pd.concat(all_dfs)

100%|██████████| 23/23 [00:14<00:00,  1.61it/s]


In [22]:
ipeds          = all_dfs.reset_index().rename(columns={'index': 'UNITID'})
ipeds_cs_stats = ipeds.groupby(['UNITID', 'year']).apply(lambda x: x[x['cs']]['CTOTALT'].sum()/x['CTOTALT'].sum())
ipeds_cs_stats = ipeds_cs_stats.dropna().reset_index().rename(columns={0: 'cs_share_comp'})

### Scorecard

In [34]:
from tqdm import tqdm 
fnames = glob(sc_url+'/Field*')

scorecards = []

for fname in tqdm(fnames): 
    df = pd.read_csv(fname)
    df['year'] = int(fname.split('/')[-1].split('_')[0][-4:])
    scorecards.append(df)

all_sc = pd.concat(scorecards)

100%|██████████| 4/4 [00:10<00:00,  2.68s/it]


In [36]:
is_cs = lambda x: (len(x)==4) and (x.startswith('11'))
all_sc['cs'] = all_sc['CIPCODE'].astype(str).apply(is_cs)

In [122]:
all_sc.columns[-40:]

Index(['BBRR2_FED_COMP_N', 'BBRR2_FED_COMP_DFLT', 'BBRR2_FED_COMP_DLNQ',
       'BBRR2_FED_COMP_FBR', 'BBRR2_FED_COMP_DFR', 'BBRR2_FED_COMP_NOPROG',
       'BBRR2_FED_COMP_MAKEPROG', 'BBRR2_FED_COMP_PAIDINFULL',
       'BBRR2_FED_COMP_DISCHARGE', 'BBRR3_FED_COMP_N', 'BBRR3_FED_COMP_DFLT',
       'BBRR3_FED_COMP_DLNQ', 'BBRR3_FED_COMP_FBR', 'BBRR3_FED_COMP_DFR',
       'BBRR3_FED_COMP_NOPROG', 'BBRR3_FED_COMP_MAKEPROG',
       'BBRR3_FED_COMP_PAIDINFULL', 'BBRR3_FED_COMP_DISCHARGE',
       'EARN_COUNT_PELL_WNE_1YR', 'EARN_PELL_WNE_MDN_1YR',
       'EARN_COUNT_NOPELL_WNE_1YR', 'EARN_NOPELL_WNE_MDN_1YR',
       'EARN_COUNT_MALE_WNE_1YR', 'EARN_MALE_WNE_MDN_1YR',
       'EARN_COUNT_NOMALE_WNE_1YR', 'EARN_NOMALE_WNE_MDN_1YR',
       'EARN_COUNT_NE_3YR', 'EARN_NE_MDN_3YR', 'EARN_COUNT_WNE_3YR',
       'EARN_CNTOVER150_3YR', 'EARN_COUNT_PELL_NE_3YR', 'EARN_PELL_NE_MDN_3YR',
       'EARN_COUNT_NOPELL_NE_3YR', 'EARN_NOPELL_NE_MDN_3YR',
       'EARN_COUNT_MALE_NE_3YR', 'EARN_MALE_NE_MDN_3YR',
  

In [124]:
col = 'IPEDSCOUNT1' # 'EARN_COUNT_PELL_NE_3YR' # 

sc_cs_stats = all_sc.groupby(['UNITID', 'year']).apply(lambda x: x[x['cs']][col].sum()/x[col].sum() if x[col].sum() > 0 else np.nan)
sc_cs_stats = sc_cs_stats.reset_index().rename(columns={0: 'cs_share_aid'})
sc_cs_stats['year'] = sc_cs_stats['year'].apply(lambda x: int('20'+str(x)[:2]))

### SC vs. IPEDs

In [43]:
merged = pd.merge(ipeds_cs_stats, sc_cs_stats, left_on=['UNITID', 'year'], right_on=['UNITID', 'year'])

In [104]:
import numpy as np 
from scipy.stats import ttest_ind

comparison          = merged.groupby('UNITID').apply(lambda x: pd.Series(ttest_ind(x['cs_share_comp'], x['cs_share_aid'])))
comparison.columns  = ['effect', 'pval']
comparison          = comparison.dropna()
comparison          = comparison[~comparison['effect'].isin([np.inf, -np.inf])]
significant         = comparison[comparison['pval']<0.05]
print(sum(significant['effect']>0), sum(significant['effect']<0))
sig_insts = significant[significant['effect']>0]

11 1053


In [106]:
sig_insts.index

Index([101365, 132709, 133702, 137209, 196051, 207564, 210076, 230171, 235343,
       235431, 421832],
      dtype='int64', name='UNITID')

In [100]:
fnames = glob(drive_url+"IPEDS_data/hdfiles/*")
all_insts = []
for fname in fnames:
    all_insts.append(pd.read_csv(fname, encoding='cp1252'))

In [101]:
all_insts = pd.concat(all_insts)

In [103]:
all_insts.shape, all_insts.drop_duplicates().shape

((135534, 160), (134981, 160))

In [109]:
all_insts.set_index('UNITID')[['INSTNM']].drop_duplicates().loc[sig_insts.index]

,INSTNM
UNITID,
101365.0,Herzing University-Birmingham
101365.0,Herzing College
101365.0,HERZING COLLEGE
132709.0,Broward College
132709.0,BROWARD COMMUNITY COLLEGE
132709.0,Broward Community College
133702.0,Florida State College at Jacksonville
133702.0,Florida Community College at Jacksonville
133702.0,FLORIDA COMMUNITY COLLEGE AT JACKSONVILLE
